In [ ]:
# @ Auther :- Hrushikesh

In [1]:
pwd

'/home/hrushikesh/work_place/pytorch'

In [2]:
import warnings 
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

In [3]:
# reading data
data=pd.read_csv('/home/hrushikesh/work_place/pytorch/data.csv')
# encoding labels
data['diagnosis'] = data['diagnosis'].astype('category').cat.codes  
# Drop unwaanted-columns
data.drop(['id','Unnamed: 32'],axis=1,inplace=True)

In [4]:
data.shape

(569, 31)

In [5]:
data.columns

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [6]:
data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [7]:
data['diagnosis'].unique()

array([1, 0], dtype=int8)

In [8]:
def load_classification_data(df,target_col,test_size):
    torch_data = np.array(df.drop(target_col,axis=1))
    torch_labels =np.array(df[target_col])
    data = np.hstack([torch_data, torch_labels.reshape(-1,1)])
    gen_names = [f"feature_{i}" for i in range(data.shape[-1])]
    col_names = gen_names
    col_names[-1] = "target"
    data = pd.DataFrame(data, columns=col_names)
    cat_col_names = [x for x in gen_names[:-1] if len(data[x].unique())<10]
    num_col_names = [x for x in gen_names[:-1] if x not in [target_col]+cat_col_names]
    test_idx = data.sample(int(test_size * len(data)), random_state=42).index
    test = data[data.index.isin(test_idx)]
    train = data[~data.index.isin(test_idx)]
    return (train, test, ["target"],cat_col_names,num_col_names)


In [9]:
train, test, target_col, cat_col_names, num_col_names= load_classification_data(data,'diagnosis',0.2)

In [10]:
train['target'].unique()

array([1., 0.])

In [11]:
# Required Library
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, NodeConfig, TabNetModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.categorical_encoders import CategoricalEmbeddingTransformer

In [12]:
# Model configuration
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=32,
    max_epochs=100,
#     gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

Training and Evaluating

In [13]:
# Fit datapoints with model
tabular_model.fit(train=train, test=test)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 0     
1 | normalizing_batch_norm | BatchNorm1d         | 60    
2 | backbone               | FeedForwardBackbone | 19.0 M
3 | output_layer           | Linear              | 1.0 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
19.0 M    Trainable params
0         Non-trainable params
19.0 M    Total params
76.028    Total estimated model params size (MB)
Global seed set to 42


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
-------------------------

LR finder stopped early after 84 steps due to diverging loss.


--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [14]:
tabular_model.evaluate(test)

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.9646017551422119}
--------------------------------------------------------------------------------


[{'test_accuracy': 0.9646017551422119}]

# Saving the Model

In [15]:
tabular_model.save_model('/home/hrushikesh/work_place/pytorch/train_model')

In [16]:
loaded_model = TabularModel.load_from_checkpoint('/home/hrushikesh/work_place/pytorch/train_model')

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [17]:
loaded_model

In [18]:
pwd

'/home/hrushikesh/work_place/pytorch'

Testing on other datasets
🌹 iris-flower dataset

In [19]:
# reading data
data2=pd.read_csv('/home/hrushikesh/work_place/pytorch/data/Iris.csv')
data2.drop(['Id'],axis=1,inplace=True)
data2['Species'] = data2['Species'].astype('category').cat.codes# encoding labels

In [20]:
train, test, target_col, cat_col_names, num_col_names  = load_classification_data(data2,'Species',0.2)

In [23]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=16,
    max_epochs=100,
    auto_select_gpus = True,
#     gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model2 = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [24]:
tabular_model2.fit(train=train, test=test)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 0     
1 | normalizing_batch_norm | BatchNorm1d         | 8     
2 | backbone               | FeedForwardBackbone | 18.9 M
3 | output_layer           | Linear              | 1.5 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.604    Total estimated model params size (MB)
Global seed set to 42


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
-------------------------

LR finder stopped early after 79 steps due to diverging loss.
Restored states from the checkpoint file at /home/hrushikesh/work_place/pytorch/lr_find_temp_model.ckpt
Learning rate set to 1.0964781961431852e-07

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 0     
1 | normalizing_batch_norm | BatchNorm1d         | 8     
2 | backbone               | FeedForwardBackbone | 18.9 M
3 | output_layer           | Linear              | 1.5 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.604    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [25]:
tabular_model2.evaluate(test)

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.699999988079071}
--------------------------------------------------------------------------------


[{'test_accuracy': 0.699999988079071}]

# pima-indian-diabetes dataset

In [26]:
# reading data
data3=pd.read_csv('/home/hrushikesh/work_place/pytorch/data/diabetes.csv')
# data3.drop('id',axis=1,inplace=True)
data3['quality'] = data3['Outcome'].astype('category').cat.codes# encoding labels

In [27]:
train, test, target_col, cat_col_names, num_col_names= load_classification_data(data3,'quality',0.2)

In [28]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=32,
    max_epochs=100,
#     gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="1024-1024-32",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model3 = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)


In [29]:
tabular_model3.fit(train=train, test=test)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 6     
1 | normalizing_batch_norm | BatchNorm1d         | 16    
2 | backbone               | FeedForwardBackbone | 1.1 M 
3 | output_layer           | Linear              | 66    
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.375     Total estimated model params size (MB)
Global seed set to 42


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------


LR finder stopped early after 96 steps due to diverging loss.
Restored states from the checkpoint file at /home/hrushikesh/work_place/pytorch/lr_find_temp_model.ckpt
Learning rate set to 6.918309709189363e-05

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 6     
1 | normalizing_batch_norm | BatchNorm1d         | 16    
2 | backbone               | FeedForwardBackbone | 1.1 M 
3 | output_layer           | Linear              | 66    
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.375     Total estimated model params size (MB)


--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [30]:
tabular_model3.evaluate(test)

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.8300653696060181}
--------------------------------------------------------------------------------


[{'test_accuracy': 0.8300653696060181}]

# heart-disease-uci dataset

In [61]:
# reading data
data4=pd.read_csv('/home/hrushikesh/work_place/pytorch/data/heart_cleveland_upload.csv')
# data3.drop('id',axis=1,inplace=True)
data4['condition'] = data4['condition'].astype('category').cat.codes# encoding labels

In [50]:
data4['condition'].unique()

array([0, 1], dtype=int8)

In [55]:
train, test, target_col, cat_col_names, num_col_names= load_classification_data(data4,'condition',0.2)

In [64]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=32,
    max_epochs=100,
#     gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model4 = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [65]:
tabular_model4.fit(train=train, test=test)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 72    
1 | normalizing_batch_norm | BatchNorm1d         | 10    
2 | backbone               | FeedForwardBackbone | 19.0 M
3 | output_layer           | Linear              | 1.0 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
19.0 M    Trainable params
0         Non-trainable params
19.0 M    Total params
75.914    Total estimated model params size (MB)
Global seed set to 42


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
-------------------------

LR finder stopped early after 88 steps due to diverging loss.
Restored states from the checkpoint file at /home/hrushikesh/work_place/pytorch/lr_find_temp_model.ckpt
Learning rate set to 0.0004365158322401656

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 72    
1 | normalizing_batch_norm | BatchNorm1d         | 10    
2 | backbone               | FeedForwardBackbone | 19.0 M
3 | output_layer           | Linear              | 1.0 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
19.0 M    Trainable params
0         Non-trainable params
19.0 M    Total params
75.914    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [72]:
# tabular_model4.evaluate(test)

# winequality-red dataset

In [36]:
# reading data
data5=pd.read_csv('/home/hrushikesh/work_place/pytorch/data/winequality-red.csv')
data5['quality'] = data5['quality'].astype('category').cat.codes# encoding labels

In [37]:
train, test, target_col, cat_col_names, num_col_names= load_classification_data(data5,'quality',0.2)

In [39]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=32,
    max_epochs=100,
#     gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model5 = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [40]:
tabular_model5.fit(train=train, test=test)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 0     
1 | normalizing_batch_norm | BatchNorm1d         | 22    
2 | backbone               | FeedForwardBackbone | 18.9 M
3 | output_layer           | Linear              | 3.1 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.725    Total estimated model params size (MB)
Global seed set to 42


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------


LR finder stopped early after 96 steps due to diverging loss.
Restored states from the checkpoint file at /home/hrushikesh/work_place/pytorch/lr_find_temp_model.ckpt


--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [41]:
tabular_model5.evaluate(test)

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.5768024921417236}
--------------------------------------------------------------------------------


[{'test_accuracy': 0.5768024921417236}]

# logs testing

In [67]:
# reading data
data6=pd.read_csv('/home/hrushikesh/Downloads/pytorch_testing.csv')
# data5['quality'] = data5['quality'].astype('category').cat.codes# encoding labels

In [1]:
# data6.columns

In [70]:
train, test, target_col, cat_col_names, num_col_names= load_classification_data(data6,'signatuire',0.2)

In [76]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=3200,
    max_epochs=100,
#     gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model6 = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [77]:
tabular_model6.fit(train=train, test=test)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 79    
1 | normalizing_batch_norm | BatchNorm1d         | 16    
2 | backbone               | FeedForwardBackbone | 19.0 M
3 | output_layer           | Linear              | 2.6 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
19.0 M    Trainable params
0         Non-trainable params
19.0 M    Total params
75.920    Total estimated model params size (MB)
Global seed set to 42


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
-------------------------

LR finder stopped early after 96 steps due to diverging loss.
Restored states from the checkpoint file at /home/hrushikesh/work_place/pytorch/lr_find_temp_model.ckpt
Learning rate set to 1.9054607179632464e-05

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 79    
1 | normalizing_batch_norm | BatchNorm1d         | 16    
2 | backbone               | FeedForwardBackbone | 19.0 M
3 | output_layer           | Linear              | 2.6 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
19.0 M    Trainable params
0         Non-trainable params
19.0 M    Total params
75.920    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [78]:
tabular_model6.evaluate(test)

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.9971320033073425}
--------------------------------------------------------------------------------


[{'test_accuracy': 0.9971320033073425}]